In [20]:
import numpy as np
import mindspore.ops as ops
import mindspore.nn as nn
from mindspore import ParameterTuple
from mindspore import context, Tensor
from mindspore.common.initializer import TruncatedNormal, Normal
from mindspore.nn import Dense, WithLossCell, SoftmaxCrossEntropyWithLogits, Momentum
from mindspore import ms_function
from mindspore import dtype as mstype

In [3]:
# Switch to PyNative mode
context.set_context(mode=context.PYNATIVE_MODE, device_target='CPU')

In [29]:
# Print the output of the Conv2D
conv = nn.Conv2d(3, 4, 3, bias_init='zeros')
input_data = Tensor(np.ones([1, 3, 5, 5]).astype(np.float32))
output = conv(input_data)
print(conv)
print(output.asnumpy())

Conv2d<input_channels=3, output_channels=4, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
[[[[ 0.0146143   0.02124701  0.02124701  0.02124701  0.01451317]
   [ 0.03753834  0.05598685  0.05598685  0.05598685  0.05175477]
   [ 0.03753834  0.05598685  0.05598685  0.05598685  0.05175477]
   [ 0.03753834  0.05598685  0.05598685  0.05598685  0.05175477]
   [ 0.03922746  0.06110519  0.06110519  0.06110519  0.04265858]]

  [[ 0.02232655 -0.01406118 -0.01406118 -0.01406118 -0.02034742]
   [-0.02411446 -0.04379661 -0.04379661 -0.04379661 -0.02431867]
   [-0.02411446 -0.04379661 -0.04379661 -0.04379661 -0.02431867]
   [-0.02411446 -0.04379661 -0.04379661 -0.04379661 -0.02431867]
   [-0.03821754 -0.05530538 -0.05530538 -0.05530538 -0.02508224]]

  [[-0.04889525 -0.08460522 -0.08460522 -0.08460522 -0.05639731]
   [-0.05491669 -0.07595994 -0.07595994 -0.07595994 -0.05406899]
   [-0.05491669 -0.

In [32]:
# Define add function
def add_func(x, y):
    z = ops.add(x, y)
    z = ops.add(z, x)
    return z

x = Tensor(np.ones([3, 3]))
y = Tensor(np.ones([3, 3], dtype=np.float32))
output = add_func(x, y)

print(output.asnumpy())

[[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]]


In [10]:
context.set_context(mode=context.PYNATIVE_MODE, device_target='CPU')

@ms_function
def add_fn(x, y):
    res = ops.add(x, y)
    return res

x = Tensor(np.ones([4, 4]).astype(np.float32))
y = Tensor(np.ones([4, 4]).astype(np.float32))
z = add_fn(x, y)

print(z.asnumpy())

[[2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]]


In [13]:
# Conv2d instantiation operation
conv_obj = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=2, padding=0)
conv_obj.init_parameters_data()

@ms_function
def conv_fn(x):
    res = conv_obj(x)
    return res

input_data = np.random.randn(2, 3, 6, 6).astype(np.float32)
z = conv_fn(Tensor(input_data))
print(z.asnumpy())

[[[[-0.00486121 -0.00426182  0.01304711]
   [-0.0401778  -0.04261138 -0.05288866]
   [ 0.05595319  0.0143291   0.01743981]]

  [[ 0.07857811 -0.04865037 -0.04082615]
   [-0.0407213   0.01885145 -0.03312718]
   [ 0.04571144  0.0049194   0.04897681]]

  [[-0.03116783 -0.06502506 -0.05381576]
   [-0.06943066 -0.05022642 -0.07986193]
   [ 0.1514199  -0.00774919 -0.00895873]]

  [[ 0.03406389 -0.04348302 -0.08774997]
   [ 0.10040696  0.04612008 -0.00733287]
   [ 0.00314185  0.03010257 -0.00499284]]]


 [[[-0.02658262 -0.0099051   0.00398355]
   [ 0.01013395 -0.03654826  0.01247858]
   [ 0.03328361  0.0030569   0.01121306]]

  [[ 0.01739827 -0.06154104  0.0163665 ]
   [-0.00363971  0.05736038  0.02763404]
   [ 0.03927846 -0.02689574  0.00300152]]

  [[ 0.00842803 -0.02577049  0.02012745]
   [-0.00594857  0.01501819 -0.02091446]
   [-0.02421508 -0.03489502  0.04148066]]

  [[ 0.02095094 -0.05682157  0.06493081]
   [-0.10339901  0.04774071 -0.06323282]
   [-0.02638944 -0.01695368 -0.02592031]]

In [14]:
def mul(x, y):
    return x * y

def mainf(x, y):
    return ops.GradOperation(get_all=True)(mul)(x, y)

x = Tensor(1, mstype.int32)
y = Tensor(2, mstype.int32)

print(mainf(x, y))

(Tensor(shape=[], dtype=Int32, value= 2), Tensor(shape=[], dtype=Int32, value= 1))


In [22]:
context.set_context(mode=context.PYNATIVE_MODE, device_target='CPU')

class LeNet5(nn.Cell):
    # LeNet network structure
    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        
    def construct(self, x):
        # Construct the forward network using defined operations
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    
# Instantiate the network
net = LeNet5()

In [23]:
class GradWrap(nn.Cell):
    # Find function input gradient
    def __init__(self, network):
        super(GradWrap, self).__init__(auto_prefix=False)
        self.network = network
        self.weights = ParameterTuple(filter(lambda x: x.requires_grad, network.get_parameters()))
        
    def construct(self, x, label):
        weights = self.weights
        return ops.GradOperation(get_by_list=True)(self.network, weights)(x, label)

In [34]:
# Set optimizer and loss function
optimizer = Momentum(filter(lambda x: x.requires_grad, net.get_parameters()), 0.1, 0.9)
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') # Get the loss value through WithLossCell

net_with_criterion = WithLossCell(net, criterion)

train_network = GradWrap(net_with_criterion)
train_network.set_train()

# Generate input data
input_data = Tensor(np.ones([32, 1, 32, 32]).astype(np.float32) * 0.01)
label = Tensor(np.ones([32]).astype(np.int32))
output = net(Tensor(input_data))

# Use forward network to calculate loss
loss_output = criterion(output, label) 
grads = train_network(input_data, label) # Optimize parameters
success = optimizer(grads)
loss = loss_output.asnumpy()

print(loss)

1.4900769
